In [2]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split

import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using Theano backend.


In [3]:
SEED = 2017
DATA_DIR = 'Data/spoken_numbers_pcm/' 

In [4]:
files = glob.glob(DATA_DIR + "*.wav")
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 0
# Validation examples: 0


In [5]:
n_features = 20
max_length = 80
n_classes = 10

In [ ]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = one_hot_encode(int(wav.split('/')[-1][0]), n_classes)
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(y)

In [ ]:
def one_hot_encode(labels_dense, n_classes=10):
    return np.eye(n_classes)[labels_dense]

In [ ]:
X_example, y_example = next(batch_generator(X_train, batch_size=1))
print('Shape of training example: {}'.format(X_example.shape))
print('Shape of training example label: {}'.format(y_example.shape))

In [ ]:
learning_rate = 0.001
batch_size = 64
n_epochs = 50
dropout = 0.5

input_shape = X_example.shape[1:]
steps_per_epoch = 50

In [ ]:
model = Sequential() model.add(LSTM(256, return_sequences=True, input_shape=input_shape, dropout=dropout)) 
model.add(Flatten()) model.add(Dense(128, activation='relu')) model.add(Dropout(dropout)) 
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
opt = Adam(lr=learning_rate) model.compile(loss='categorical_crossentropy', optimizer=opt, 
metrics=['accuracy']) model.summary()

In [ ]:
history = model.fit_generator(
 generator=batch_generator(X_train, batch_size),
 steps_per_epoch=steps_per_epoch,
 epochs=n_epochs,
 verbose=1,
 validation_data=batch_generator(X_val, 32),
 validation_steps=5
 )